In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd

In [2]:
df = pd.read_csv('celeb_attr.csv', index_col=False)
df = df.replace(-1, 0)

In [3]:
class CelebrityImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        
        img_path = f"{self.img_dir}/{self.dataframe.iloc[idx]['Image_Name']}"
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        labels = torch.tensor(self.dataframe.iloc[idx][1:].values.astype('float32'))

        return image, labels

In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [5]:
from torch.utils.data import DataLoader

In [136]:
import torch.nn as nn
import torch.nn.functional as F

class AttributeCNN(nn.Module):
    def __init__(self, num_classes=40):
        super(AttributeCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.relu3 = nn.ReLU()

        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128*16*16, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.pool3(x)
        x = self.relu3(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))

        return x




In [137]:
import torch.optim as optim

device= torch.device("cuda")

model = AttributeCNN(num_classes=40)
model.to(device)

evaluator = nn.BCELoss()
optimizer= optim.Adam(model.parameters(), lr=0.001)

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = CelebrityImageDataset(train_df, img_dir='datasets/celeba/img_celeba/img_celeba', transform=transform)
test_dataset = CelebrityImageDataset(test_df, img_dir='datasets/celeba/img_celeba/img_celeba', transform=transform)

In [9]:
train_loading = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loading = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4)

In [14]:
epochs = 10
from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss= 0.0

    loop = tqdm(train_loading, desc=f"Epoch [{epoch+1}/{epochs}]")
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()

        outputs = model(images) #forward pass
        loss = evaluator(outputs, labels)
        optimizer.zero_grad() #Zero Gradient
        loss.backward() # Backprop

        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss/len(train_loading)
    print(f"Epoch {epoch+1}, Loss {avg_loss:.4f}")

Epoch [1/10]: 100%|██████████| 5065/5065 [01:48<00:00, 46.65it/s, loss=0.32] 


Epoch 1, Loss 0.2974


Epoch [2/10]: 100%|██████████| 5065/5065 [01:47<00:00, 47.10it/s, loss=0.287]


Epoch 2, Loss 0.2874


Epoch [3/10]: 100%|██████████| 5065/5065 [01:46<00:00, 47.73it/s, loss=0.297]


...

Epoch 8, Loss 0.2508


Epoch [9/10]: 100%|██████████| 5065/5065 [01:44<00:00, 48.61it/s, loss=0.234]


Epoch 9, Loss 0.2464


Epoch [10/10]: 100%|██████████| 5065/5065 [01:43<00:00, 48.80it/s, loss=0.237]

Epoch 10, Loss 0.2424


In [138]:
model = AttributeCNN()
model.load_state_dict(torch.load('models/Model1.pth', weights_only=True))
model.eval()


AttributeCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu3): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=32768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=40, bias=True)
)

In [ ]:
# Prediction Code

from torchvision import transforms
from PIL import Image

model = AttributeCNN()
model.load_state_dict(torch.load('models/Model1.pth', weights_only=True))
model.eval()


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

image = Image.open('image.jpg').convert("RGB")

image_tensor = transform(image).unsqueeze(0)


model.eval()
model.to(device)
image_tensor = image_tensor.to(device)

with torch.no_grad():
    output = model(image_tensor)


    preds = (output > 0.5).int() 

preds

tensor([[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
         1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1]], device='cuda:0',
       dtype=torch.int32)

In [21]:
torch.save(model.state_dict(), "models/Model1.pth")

In [24]:

from sklearn.metrics import classification_report
all_predict = []
all_targets = []
model.eval()

with torch.no_grad():
    loop = tqdm(test_loading)
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()

        outputs = model(images)
        preds = (outputs > 0.5).float()
        all_predict.append(preds.cpu())
        all_targets.append(labels.cpu())
    
all_predict = torch.cat(all_predict, dim=0).numpy()
all_targets = torch.cat(all_targets, dim=0).numpy()

100%|██████████| 1267/1267 [00:26<00:00, 48.60it/s]


In [59]:
report=classification_report(all_targets, all_predict, output_dict=True)
print(report)

{'0': {'precision': 0.6523364485981309, 'recall': 0.2341757995974055, 'f1-score': 0.3446346280447663, 'support': 4471.0}, '1': {'precision': 0.6084782090974041, 'recall': 0.4721862871927555, 'f1-score': 0.5317377731529657, 'support': 10822.0}, '2': {'precision': 0.7814687087939953, 'recall': 0.744324219882137, 'f1-score': 0.7624443344878773, 'support': 20702.0}, '3': {'precision': 0.5866369710467706, 'recall': 0.1571974218190499, 'f1-score': 0.24795255577520475, 'support': 8378.0}, '4': {'precision': 0.6938775510204082, 'recall': 0.33663366336633666, 'f1-score': 0.4533333333333333, 'support': 909.0}, '5': {'precision': 0.7675753228120517, 'recall': 0.5130254115390762, 'f1-score': 0.6150014369192451, 'support': 6257.0}, '6': {'precision': 0.5695674830640959, 'recall': 0.11161033391197794, 'f1-score': 0.18664617486338797, 'support': 9793.0}, '7': {'precision': 0.6518501279962765, 'recall': 0.29283847360167276, 'f1-score': 0.4041263886885009, 'support': 9565.0}, '8': {'precision': 0.73680

In [60]:
classification_df = pd.DataFrame(report).transpose()


In [74]:
individual_stats = classification_df[:-4].set_index(df.columns[1:])
summary_stats = classification_df.iloc[-4:]

In [77]:
individual_stats.to_csv("individual_stats1.csv")
summary_stats.to_csv("summary_stats1.csv")

In [26]:
df.sum()

Image_Name             000001.jpg000002.jpg000003.jpg000004.jpg000005...
5_o_Clock_Shadow                                                   22516
Arched_Eyebrows                                                    54090
Attractive                                                        103833
Bags_Under_Eyes                                                    41446
Bald                                                                4547
Bangs                                                              30709
Big_Lips                                                           48785
Big_Nose                                                           47516
Black_Hair                                                         48472
Blond_Hair                                                         29983
Blurry                                                             10312
Brown_Hair                                                         41572
Bushy_Eyebrows                                     

In [80]:
positive_counts = df.iloc[:, 1:].sum(axis=0).values
negative_counts = len(df) - positive_counts

pos_weight = negative_counts / (positive_counts + 1e-6)

In [82]:
pos_weight_tensor = torch.tensor(pos_weight, dtype=torch.float32).to(device)
evaluator = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
optimizer= optim.Adam(model.parameters(), lr=0.001)

In [83]:
epochs = 10
from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss= 0.0

    loop = tqdm(train_loading, desc=f"Epoch [{epoch+1}/{epochs}]")
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()

        outputs = model(images) #forward pass
        loss = evaluator(outputs, labels)
        optimizer.zero_grad() #Zero Gradient
        loss.backward() # Backprop

        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss/len(train_loading)
    print(f"Epoch {epoch+1}, Loss {avg_loss:.4f}")

Epoch [1/10]: 100%|██████████| 5065/5065 [02:05<00:00, 40.34it/s, loss=0.938]


Epoch 1, Loss 0.9074


Epoch [2/10]: 100%|██████████| 5065/5065 [01:46<00:00, 47.61it/s, loss=0.903]


Epoch 2, Loss 0.9041


Epoch [3/10]: 100%|██████████| 5065/5065 [01:44<00:00, 48.35it/s, loss=0.923]


...

Epoch 8, Loss 0.9130


Epoch [9/10]: 100%|██████████| 5065/5065 [01:43<00:00, 48.80it/s, loss=0.897]


Epoch 9, Loss 0.9155


Epoch [10/10]: 100%|██████████| 5065/5065 [01:43<00:00, 48.88it/s, loss=0.927]

Epoch 10, Loss 0.9178


In [ ]:

from sklearn.metrics import classification_report
all_predict = []
all_targets = []
model.eval()

with torch.no_grad():
    loop = tqdm(test_loading)
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()

        outputs = model(images)
        preds = (outputs > 0.5).float()
        all_predict.append(preds.cpu())
        all_targets.append(labels.cpu())
    
all_predict = torch.cat(all_predict, dim=0).numpy()
all_targets = torch.cat(all_targets, dim=0).numpy()

100%|██████████| 1267/1267 [00:30<00:00, 42.01it/s]


In [86]:
report=classification_report(all_targets, all_predict, output_dict=False)
print(report)

              precision    recall  f1-score   support

           0       0.36      0.68      0.47      4471
           1       0.53      0.66      0.59     10822
           2       0.82      0.58      0.68     20702
           3       0.48      0.41      0.44      8378
           4       0.16      0.86      0.27       909
           5       0.62      0.55      0.58      6257
           6       0.41      0.35      0.38      9793
           7       0.53      0.47      0.50      9565
           8       0.60      0.77      0.67      9649
           9       0.65      0.81      0.72      6023
          10       0.15      0.58      0.24      1953
          11       0.66      0.50      0.57      8465
          12       0.47      0.52      0.49      5775
          13       0.23      0.66      0.34      2331
          14       0.21      0.65      0.32      1895
          15       0.32      0.77      0.45      2588
          16       0.32      0.72      0.45      2536
          17       0.27    

/home/skrubstar/Datahacks/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [104]:
from sklearn.metrics import classification_report
all_probabilities = []
all_targets = []
model.eval()

with torch.no_grad():
    loop = tqdm(test_loading)
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()

        outputs = model(images)
        probabilities = torch.sigmoid(outputs)

        all_probabilities.append(probabilities.cpu())
        all_targets.append(labels.cpu())
    
all_predict = torch.cat(all_probabilities, dim=0).numpy()
all_targets = torch.cat(all_targets, dim=0).numpy()

100%|██████████| 1267/1267 [00:25<00:00, 48.92it/s]


In [105]:
def compute_optimal_thresholds(y_true, y_probs):
    thresholds = []
    for i in range(y_true.shape[1]):
        precision, recall, threshold = precision_recall_curve(y_true[:, i], y_probs[:, i])
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        if len(threshold) == 0:
            thresholds.append(0.5)
        else:
            best_thresh = threshold[np.argmax(f1)]
            thresholds.append(best_thresh)
    return np.array(thresholds)

In [107]:
optimal_thresholds = compute_optimal_thresholds(all_targets, all_predict) #Classes

In [108]:
optimal_thresholds

array([0.7103994 , 0.5317982 , 0.50000083, 0.5031747 , 0.73105854,
       0.5670702 , 0.5000152 , 0.50071186, 0.5612252 , 0.71674156,
       0.7225935 , 0.5008443 , 0.6052067 , 0.73105836, 0.7310523 ,
       0.73105854, 0.7310584 , 0.73105854, 0.513892  , 0.50000197,
       0.536272  , 0.50000024, 0.7310581 , 0.50038797, 0.50000006,
       0.5000585 , 0.72613126, 0.5002488 , 0.7310583 , 0.73105556,
       0.7310584 , 0.50000167, 0.51433396, 0.50000674, 0.5354066 ,
       0.7310583 , 0.5024743 , 0.5459233 , 0.73105854, 0.50000006],
      dtype=float32)

In [111]:
all_predict = []
all_targets = []
model.eval()

thresholds = torch.tensor(optimal_thresholds).to(device)

with torch.no_grad():
    loop = tqdm(test_loading)
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()

        outputs = model(images)
        probabilities = torch.sigmoid(outputs)

        predictions = (probabilities > thresholds.unsqueeze(0)).float()

        all_predict.append(predictions.cpu())
        all_targets.append(labels.cpu())
    
all_predict = torch.cat(all_predict, dim=0).numpy()
all_targets = torch.cat(all_targets, dim=0).numpy()

100%|██████████| 1267/1267 [00:25<00:00, 49.74it/s]


In [112]:
report=classification_report(all_targets, all_predict, output_dict=False)
print(report)

              precision    recall  f1-score   support

           0       0.38      0.63      0.48      4471
           1       0.51      0.71      0.59     10822
           2       0.69      0.84      0.76     20702
           3       0.41      0.54      0.47      8378
           4       0.00      0.00      0.00       909
           5       0.60      0.58      0.59      6257
           6       0.33      0.72      0.45      9793
           7       0.45      0.62      0.52      9565
           8       0.58      0.79      0.67      9649
           9       0.69      0.78      0.73      6023
          10       0.18      0.48      0.26      1953
          11       0.54      0.64      0.59      8465
          12       0.46      0.53      0.49      5775
          13       0.31      0.47      0.38      2331
          14       0.27      0.52      0.36      1895
          15       0.00      0.00      0.00      2588
          16       0.50      0.53      0.51      2536
          17       0.00    

/home/skrubstar/Datahacks/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
